<a href="https://colab.research.google.com/github/rlatjdalsl/Phishing-Block/blob/main/stt_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install SpeechRecognition pydub
!apt-get install -y ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 24.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
import speech_recognition as sr
import os

# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# VoiceFile 폴더 경로 설정
voice_file_path = '/content/drive/My Drive/Colab Notebooks/VoiceFile'

# VoiceFile 폴더 안의 파일 목록 확인
file_list = os.listdir(voice_file_path)
print("VoiceFile 폴더 안의 파일 목록:")
for file in file_list:
    print(file)

def google_stt_from_drive(file_name):
    recognizer = sr.Recognizer()
    file_path = f"{voice_file_path}/{file_name}"
    try:
        with sr.AudioFile(file_path) as source:
            print("음성을 불러오는 중...")
            audio = recognizer.record(source)
            print("Google STT 변환 중...")
            text = recognizer.recognize_google(audio, language="ko-KR")
            print("인식된 텍스트:", text)
            return text
    except sr.RequestError:
        print("API 오류: Google 서버에 접근할 수 없습니다.")
    except sr.UnknownValueError:
        print("음성을 인식하지 못했습니다.")
    return None

# 첫 번째 파일로 테스트
if file_list:
    file_name = file_list[0]  # VoiceFile 폴더 안의 첫 번째 파일 사용
    google_stt_from_drive(file_name)
else:
    print("VoiceFile 폴더에 음성 파일이 없습니다.")


Mounted at /content/drive
VoiceFile 폴더 안의 파일 목록:
test.wav
model
phishing
normal
음성을 불러오는 중...
Google STT 변환 중...
인식된 텍스트: 안녕하세요 하나 둘 셋 하나 둘 셋 저는 사기꾼입니다 보이스 피싱 저는 보이스피싱범입니다 돈을 빨리 보내 주셔야 지금 아들이 저랑 같이 있어요 지금 딸이 저랑 같이 있어요 지금 어머니가 크게 다치셨어요 지금 아버지가 크게 다치셨어요 안녕하세요 검찰 지금 대포 통장이 연료 되셨는데요


In [ ]:
import os
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Google Drive 경로 설정
base_path = '/content/drive/My Drive/Colab Notebooks/VoiceFile'
normal_path = f'{base_path}/normal'
phishing_path = f'{base_path}/phishing'

# 텍스트 파일에서 문장 읽기
def load_texts_from_folder(folder_path, label):
    texts = []
    labels = []
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if file_name.endswith('.txt'):
            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read().strip()
                texts.append(text)
                labels.append(label)
    return texts, labels

# 정상 대화 데이터 로드
normal_texts, normal_labels = load_texts_from_folder(normal_path, label=0)

# 보이스피싱 의심 대화 데이터 로드
phishing_texts, phishing_labels = load_texts_from_folder(phishing_path, label=1)

# 모든 데이터를 하나로 합치기
texts = normal_texts + phishing_texts
labels = normal_labels + phishing_labels

# 데이터 확인
print(f"총 데이터 개수: {len(texts)}")
print(f"일반 대화: {len(normal_texts)}, 보이스피싱 대화: {len(phishing_texts)}")

# 토크나이저 생성 및 학습
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(texts)

# 시퀀스 변환
sequences = tokenizer.texts_to_sequences(texts)
padded = pad_sequences(sequences, maxlen=200)

# numpy 배열로 변환
labels = np.array(labels)

print("✅ 데이터 로드 및 전처리 완료!")


총 데이터 개수: 15
일반 대화: 7, 보이스피싱 대화: 8
✅ 데이터 로드 및 전처리 완료!


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# 모델 정의
model = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_length=100),
    LSTM(64),
    Dense(1, activation='sigmoid')
])

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(padded, labels, epochs=5, batch_size=8)

# 모델 저장
model_path = f'{base_path}/model'
if not os.path.exists(model_path):
    os.makedirs(model_path)

model.save(f"{model_path}/phishing_model.h5")

# 토크나이저 저장
import pickle
with open(f"{model_path}/tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

print("✅ 모델 학습 및 저장 완료!")


Epoch 1/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - accuracy: 0.8250 - loss: 0.6895
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.9139 - loss: 0.6775
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 1.0000 - loss: 0.6643
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 1.0000 - loss: 0.6461
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 1.0000 - loss: 0.6257


✅ 모델 학습 및 저장 완료!


Mounted at /content/drive


In [ ]:
import speech_recognition as sr
import os
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 경로 설정
voice_file_path = '/content/drive/My Drive/Colab Notebooks/VoiceFile'
model_path = f"{voice_file_path}/model/phishing_model.h5"
tokenizer_path = f"{voice_file_path}/model/tokenizer.pkl"

# 모델과 토크나이저 로드
print("🔁 모델과 토크나이저 로딩 중...")
model = load_model(model_path)
with open(tokenizer_path, 'rb') as f:
    tokenizer = pickle.load(f)
print("✅ 모델 로딩 완료!")

def analyze_audio_risk(file_name, threshold=0.5):
    recognizer = sr.Recognizer()
    file_path = os.path.join(voice_file_path, file_name)

    # 1. STT 변환
    try:
        with sr.AudioFile(file_path) as source:
            print(f"🎧 음성 파일 불러오는 중: {file_name}")
            audio = recognizer.record(source)
            print("🔁 Google STT 변환 중...")
            text = recognizer.recognize_google(audio, language="ko-KR")
            print("📝 인식된 텍스트:", text)
    except sr.UnknownValueError:
        print("❌ 음성을 인식하지 못했습니다.")
        return
    except sr.RequestError:
        print("❌ Google STT API 오류 발생.")
        return

    # 2. Keras 위험도 분석
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=100)
    prediction = model.predict(padded)[0][0]

    print(f"📊 위험도 예측값: {prediction:.4f}")
    if prediction > threshold:
        print("🚨 보이스피싱 가능성 높음!")
    else:
        print("✅ 정상 대화로 판단됨")

    # 3. LLM 분석 추가
    llm_result = llm_analysis(text, prediction, threshold)
    print("💬 최종 판단:", llm_result)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🔁 모델과 토크나이저 로딩 중...
✅ 모델 로딩 완료!


In [ ]:
from openai import OpenAI

# OpenRouter API 설정
client = OpenAI(
    api_key="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",  # OpenRouter에서 발급받은 API 키 입력
    base_url="https://openrouter.ai/api/v1"
)

# LLM 분석 함수 정의
def llm_analysis(text, risk_score, threshold=0.5):
    print(f"\n📊 위험도: {risk_score:.4f}")
    if risk_score <= threshold:
        print("✅ 위험도 낮아 LLM 분석 생략")
        return "정상 대화로 간주"

    print("🚨 위험도 높음 → LLM 분석 시작")

    try:
        response = client.chat.completions.create(
            model="tngtech/deepseek-r1t-chimera:free",  # DeepSeek Chimera 모델 지정
            messages=[
                {
                    "role": "system",
                    "content": (
                        "당신은 보이스피싱 탐지 전문가입니다. "
                        "아래 문장이 보이스피싱인지 판단해 주세요. "
                        "보이스피싱의 기준은 다음과 같습니다"
                        "- 금전 요구(송금, 입금, 대출, 투자 권유 등)"
                        "- 긴박한 상황 조성(자녀 사고, 검찰·경찰 사칭, 계좌 정지, 개인정보 노출 등)"
                        "- 전화나 문자 등으로 개인정보, 금융정보를 요구하거나 조작된 링크 클릭을 유도"
                        "- 말투나 단어에서 불안, 협박, 회유, 급박함이 느껴질 경우"
                        "- 감정적으로 압박하며 죄책감을 유도하는 경우"
                        "- 금전을 빌려달라는 요구를 통해 상대방을 공범으로 만들려는 경우가 있는경우"
                        "해당 문장이 위 기준에 부합하면 “보이스피싱입니다”, 그렇지 않으면 “정상 대화입니다” 라고 **딱 한 문장만 출력**하십시오."
                    )
                },
                {
                    "role": "user",
                    "content": text
                }
            ]
        )

        result = response.choices[0].message.content.strip()
        print("📢 LLM 분석 결과:", result)
        return result

    except Exception as e:
        print("❌ LLM 분석 중 오류 발생:", str(e))
        return "분석 실패"

# VoiceFile 폴더 내 파일 목록 확인
file_list = os.listdir(voice_file_path)
print("📂 VoiceFile 폴더 안의 파일 목록:")
for file in file_list:
    print("-", file)

# 첫 번째 파일로 분석 테스트 실행
if file_list:
    test_file = file_list[0]
    analyze_audio_risk(test_file)
else:
    print("❗ VoiceFile 폴더에 분석할 음성 파일이 없습니다.")


📂 VoiceFile 폴더 안의 파일 목록:
- test.wav
- model
- phishing
- normal
🎧 음성 파일 불러오는 중: test.wav
🔁 Google STT 변환 중...
📝 인식된 텍스트: 안녕하세요 하나 둘 셋 하나 둘 셋 저는 사기꾼입니다 보이스 피싱 저는 보이스피싱범입니다 돈을 빨리 보내 주셔야 지금 아들이 저랑 같이 있어요 지금 딸이 저랑 같이 있어요 지금 어머니가 크게 다치셨어요 지금 아버지가 크게 다치셨어요 안녕하세요 검찰 지금 대포 통장이 연료 되셨는데요
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
📊 위험도 예측값: 0.0970
✅ 정상 대화로 판단됨

📊 위험도: 0.0970
✅ 위험도 낮아 LLM 분석 생략
💬 최종 판단: 정상 대화로 간주
